# Import

In [1]:
import codecs
from pythainlp.tokenize import subword_tokenize
from pythainlp.tokenize import tcc
#import deepcut
from pythainlp.tag import pos_tag
from nltk.tokenize import RegexpTokenizer
import glob
import nltk
import re
# thai cut
thaicut="newmm"
from sklearn_crfsuite import scorers,metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate,train_test_split
import sklearn_crfsuite
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import  thai_syllables as syllable_dict
stopwords = list(thai_stopwords())

# จัดการ BBcode

In [2]:
data_not=[]
def Unique(p):
 global data_not
 text=re.sub("<[^>]*>","",p)
 text=re.sub("\[(.*?)\]","",text)
 text=re.sub("\[\/(.*?)\]","",text)
 if text not in data_not:
  data_not.append(text)
  return True
 else:
  return False

In [3]:
# เตรียมตัวตัด tag ด้วย re
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น ('TIME','ไง','TIME')

In [4]:
# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
 text=text.strip().replace("FACILITY","LOCATION").replace("[AGO]","").replace("[/AGO]","").replace("[T]","").replace("[/T]","")
 text=re.sub("<[^>]*>","",text)
 text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')# text.replace('>','>***') # ตัดการกับพวกไม่มี tag word
 text=re.sub("(\[\w+\])","***\\1",text)
 text2=[]
 for i in text.split('***'):
  if "[" in i:
   text2.append(i)
  else:
   text2.append("[word]"+i+"[/word]")
 text="".join(text2)#re.sub("[word][/word]","","".join(text2))
 return text.replace("[word][/word]","")

In [5]:
# แปลง text ให้เป็น conll2002
def text2conll2002(text,pos=True):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text)
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer.tokenize(text)
    j=0
    conll2002=""
    for tagopen,text,tagclose in tag:
        word_cut=subword_tokenize(text, engine="dict")#,engine=thaicut) # ใช้ตัวตัดคำ newmm
        i=0
        txt5=""
        while i<len(word_cut):
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen
            else:
                txt5+=word_cut[i]
                txt5+='\t'+'O'
            txt5+='\n'
            #j+=1
            i+=1
        conll2002+=txt5
    if pos==False:
        return conll2002
    return conll2002

In [6]:
# เขียนไฟล์ข้อมูล conll2002
def write_conll2002(file_name,data):
    """
    ใช้สำหรับเขียนไฟล์
    """
    with codecs.open(file_name, "w", "utf-8-sig") as temp:
        temp.write(data)
    return True
# อ่านข้อมูลจากไฟล์
def get_data(fileopen):
	"""
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
	with codecs.open(fileopen, 'r',encoding='utf-8-sig') as f:
		lines = f.read().splitlines()
	return [a for a in lines if Unique(a)] # เอาไม่ซ้ำกัน

def alldata(lists):
    text=""
    for data in lists:
        text+=text2conll2002(data)
        text+='\n'
    return text

def alldata_list(lists):
    data_all=[]
    for data in lists:
        data_num=[]
        try:
            txt=text2conll2002(data,pos=True).split('\n')
            for d in txt:
                tt=d.split('\t')
                if d!="":
                    if len(tt)==3:
                        data_num.append((tt[0],tt[1],tt[2]))
                    else:
                        data_num.append((tt[0],tt[1]))
            #print(data_num)
            data_all.append(data_num)
        except:
            print(data)
    #print(data_all)
    return data_all

def alldata_list_str(lists):
	string=""
	for data in lists:
		string1=""
		for j in data:
			string1+=j[0]+"	"+j[1]+"	"+j[2]+"\n"
		string1+="\n"
		string+=string1
	return string

def get_data_tag(listd):
	list_all=[]
	c=[]
	for i in listd:
		if i !='':
			c.append((i.split("\t")[0],i.split("\t")[1],i.split("\t")[2]))
		else:
			list_all.append(c)
			c=[]
	return list_all
def getall(lista):
    ll=[]
    for i in lista:
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(i)
    return ll

# Load data

In [7]:
data=getall(get_data("data.txt"))
print(len(data))

1785


In [8]:
datatofile=alldata_list(data)

In [9]:
datatofile[0]

[('ได้', 'O'),
 ('ข้อ', 'O'),
 ('มูล', 'O'),
 ('ที่', 'O'),
 ('มี', 'O'),
 ('ป', 'B-S'),
 ('ระ', 'I-S'),
 ('โย', 'I-S'),
 ('ด', 'I-S'),
 ('มา', 'O'),
 ('คง', 'O'),
 ('ดี', 'O'),
 ('นะ', 'O'),
 ('ครับ', 'O')]

# คุณสมบัติ

In [10]:
import string
invalidChars = set(string.punctuation.replace("_", ""))
dict_s=list(set(syllable_dict()))
def c(word):
    for i in list('กขฃคฆงจชซญฎฏฐฑฒณดตถทธนบปพฟภมยรลวศษสฬอ'):
        if i in word:
            return True
    return False
def n(word):
    for i in list('ฅฉผฟฌหฮ'):
        if i in word:
            return True
    return False
def v(word):
    for i in list('ะาำิีืึุู'):
        if i in word:
            return True
    return False
def w(word):
    for i in list('เแโใไ'):
        if i in word:
            return True
    return False
def is_special_characters(w):
    if any(char in invalidChars for char in w):
        return True
    else:
        return False
def is_numthai(w):
    return w in list("๑๒๓๔๕๖๗๘๙๐")
def lenbytcc(w):
    return tcc.tcc(w, sep="|=/=|").count('|=/=|')
def in_dict(word):
    return word in dict_s
def has_silencer(word):
    for i in list('์ๆฯ.'):
        if i in word:
            return True
    return False
def has_tonemarks(word):
    t=False
    for i in ['่','้','็','๊','๋']:
        if i in word:
            t=True
    return t
def isThai(chr):
 cVal = ord(chr)
 if(cVal >= 3584 and cVal <= 3711):
  return True
 return False
def isThaiWord(word):
 t=True
 for i in word:
  l=isThai(i)
  if l!=True and i!='.':
   t=False
   break
 return t

def is_stopword(word):
    return word in stopwords
def is_s(word):
    if word == " " or word =="\t" or word=="" or word=="\r\n" or word=="\n":
        return True
    else:
        return False

def lennum(word,num):
    if len(word)==num:
        return True
    return False
def doc2features(doc, i):
    word = doc[i][0]
    # Features from current word
    features={
        'word.word': word,
        'word.stopword': is_stopword(word),
        'word.isthai':isThaiWord(word),
        'word.isnumthai':is_numthai(word),
        'word.isspace':word.isspace(),
        'word.tonemarks':has_tonemarks(word),
        'word.in_dict':in_dict(word),
        'word.silencer':has_silencer(word),
        'word.isdigit': word.isdigit(),
        'word.lentcc':lenbytcc(word),
        'word.c':c(word),
        'word.n':n(word),
        'word.v':v(word),
        'word.w':w(word),
        'word.is_special_characters':is_special_characters(word)
    }
    if i > 0:
        prevword = doc[i-1][0]
        features['word.prevword'] = prevword
        features['word.previsspace']=prevword.isspace()
        features['word.previsthai']=isThaiWord(prevword)
        features['word.prevstopword']=is_stopword(prevword)
        features['word.prevtonemarks']=has_tonemarks(prevword)
        features['word.previn_dict']=in_dict(prevword)
        features['word.previn_isnumthai']=is_numthai(prevword)
        features['word.prevsilencer']=has_silencer(prevword)
        features['word.prevwordisdigit'] = prevword.isdigit()
        features['word.prevlentcc'] = lenbytcc(prevword)
        features['word.prev_c'] = c(prevword)
        features['word.prev_n'] = n(prevword)
        features['word.prev_w'] = w(prevword)
        features['word.prev_v'] = v(prevword)
        features['word.prev_is_special_characters'] =is_special_characters(prevword)
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        features['word.nextword'] = nextword
        features['word.next_isspace']=nextword.isspace()
        features['word.next_isthai']=isThaiWord(nextword)
        features['word.next_tonemarks']=has_tonemarks(nextword)
        features['word.next_stopword']=is_stopword(nextword)
        features['word.next_in_dict']=in_dict(nextword)
        features['word.next_in_isnumthai']=is_numthai(nextword)
        features['word.next_silencer']=has_silencer(nextword)
        features['word.next_wordisdigit'] = nextword.isdigit()
        features['word.next_lentcc']=lenbytcc(nextword)
        features['word.next_c']=c(nextword)
        features['word.next_n']=n(nextword)
        features['word.next_w']=w(nextword)
        features['word.next_v']=v(nextword)
        features['word.next_is_special_characters']=is_special_characters(nextword)
    else:
        features['EOS'] = True # Special "End of Sequence" tag

    return features

def extract_features(doc):
    return [doc2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [tag for (token,tag) in doc]

In [102]:
import random
random.shuffle(datatofile)

In [103]:
datatofile[0]

[('ควร', 'O'),
 ('ให้', 'O'),
 ('ครู', 'O'),
 ('ภา', 'O'),
 ('ษา', 'O'),
 ('ไทย', 'O'),
 ('กรอง', 'O'),
 ('คำ', 'O'),
 ('ก่อน', 'O'),
 ('เน', 'O'),
 ('อะ', 'O')]

# เตรียมข้อมูลสำหรับ Train

In [104]:
X_data = [extract_features(doc) for doc in datatofile]
y_data = [get_labels(doc) for doc in datatofile]

In [193]:
test=False

In [194]:
!rm test.model0

In [195]:
!rm sp.model

rm: cannot remove 'sp.model': No such file or directory


In [196]:
if test:
    X, X_test, y, y_test = train_test_split(X_data, y_data, test_size=0.1)
else:
    X=X_data
    y=y_data

# Train

In [197]:
filename=""
if test:
    filename="test.model0"
else:
    filename="sp.model"
crf = sklearn_crfsuite.CRF(
    algorithm='pa',
    #c1=0.1,
    #c2=0.1,
    max_iterations=450,#500,
    all_possible_transitions=True,
    model_filename=filename
)
crf.fit(X, y);

In [198]:
#X_test[0]

# Test

In [199]:
if test:
    labels = list(crf.classes_)
    labels.remove('O')
    y_pred = crf.predict(X_test)
    e=metrics.flat_f1_score(y_test, y_pred,average='weighted', labels=labels)
    print(e)
    sorted_labels = sorted(labels,key=lambda name: (name[1:], name[0]))
    print(metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))
else:
    labels = list(crf.classes_)
    labels.remove('O')
    y_pred = crf.predict(X)
    e=metrics.flat_f1_score(y, y_pred,average='weighted', labels=labels)
    print(e)
    sorted_labels = sorted(labels,key=lambda name: (name[1:], name[0]))
    print(metrics.flat_classification_report(y, y_pred, labels=sorted_labels, digits=3))

0.9980599668069841
             precision    recall  f1-score   support

        B-S      0.998     0.997     0.998      1105
        I-S      0.998     0.999     0.998       957

avg / total      0.998     0.998     0.998      2062



In [200]:
"""
a=0.0
b=0.0
i=0
while i<1000:
    crf = sklearn_crfsuite.CRF(
        algorithm='ap',#'lbfgs',
        #c1=0.1,
        #c2=0.1,
        max_iterations=i,
        all_possible_transitions=True,
        model_filename="test.model0."+str(i)
    )
    print(i)
    crf.fit(X, y);
    labels = list(crf.classes_)
    labels.remove('O')
    y_pred = crf.predict(X_test)
    e=metrics.flat_f1_score(y_test, y_pred,average='weighted', labels=labels)
    print(e)
    print("-----------")
    i+=50
"""

'\na=0.0\nb=0.0\ni=0\nwhile i<1000:\n    crf = sklearn_crfsuite.CRF(\n        algorithm=\'ap\',#\'lbfgs\',\n        #c1=0.1,\n        #c2=0.1,\n        max_iterations=i,\n        all_possible_transitions=True,\n        model_filename="test.model0."+str(i)\n    )\n    print(i)\n    crf.fit(X, y);\n    labels = list(crf.classes_)\n    labels.remove(\'O\')\n    y_pred = crf.predict(X_test)\n    e=metrics.flat_f1_score(y_test, y_pred,average=\'weighted\', labels=labels)\n    print(e)\n    print("-----------")\n    i+=50\n'

### cross_validate

In [201]:
if test:
    f1_scorer = make_scorer(metrics.flat_f1_score, average='macro') 
    scores = cross_validate(crf, X, y, scoring=f1_scorer, cv=5)
    # save data
    print(scores)

In [202]:
if test:
    f1_scorer = make_scorer(metrics.flat_f1_score, average='macro') 
    scores = cross_validate(crf, X_data, y_data, scoring=f1_scorer, cv=5)
    # save data
    print(scores)

In [203]:
def get(text):
    word_cut=subword_tokenize(text, engine="dict")
    #print(word_cut)
    X_test = extract_features([(i,) for i in word_cut])
    #print(X_test)
    y_=crf.predict_single(X_test)
    x= [(word_cut[i],data) for i,data in enumerate(y_)]
    output=""
    temp=''
    #print(x)
    for i,b in enumerate(x):
        if i==len(x)-1 and 'B' in b[1] and temp=='B':
            output+="</คำผิด><คำผิด>"+b[0]+"</คำผิด>"
            temp='B'
        elif i==len(x)-1 and 'B' in b[1]:
            output+="<คำผิด>"+b[0]+"</คำผิด>"
            temp='B'
        elif 'B-' in b[1] and temp=='B':
            output+="</คำผิด><คำผิด>"+b[0]
            temp='B'
        elif 'B-' in b[1]:
            output+="<คำผิด>"+b[0]
            temp='B'
        elif 'O' in b[1] and temp=='B':
            output+="</คำผิด>"+b[0]
            temp='O'
        elif i==len(x)-1 and 'I' in b[1] and temp=='B':
            output+=b[0]+"</คำผิด>"
            temp='O'
        else:
            output+=b[0]
    return output


In [204]:
get("ทดสอบ")

'ทดสอบ'

In [205]:
get("ตอนนี้คุณทดสอบระบบได้เลยนะค่ะ")

'ตอนนี้คุณทดสอบระบบได้เลย<คำผิด>นะค่ะ</คำผิด>'

In [206]:
get("ผมจะบกลศสำคัญตอนนี้คุณทดสอบระบบได้เลยนะค่ะ")

'ผมจะ<คำผิด>บกลศ</คำผิด>สำคัญตอนนี้คุณทดสอบระบบได้เลย<คำผิด>นะค่ะ</คำผิด>'

In [207]:
get("กระเป๋ามันคับจริงๆ")

'กระเป๋ามันคับจริงๆ'

In [208]:
get("มข. เงียบกว่าครับ ผมไม่รู้ความเคลื่อนไหวอะไรเลยในด้านนี้ครับ")

'มข. เงียบกว่าครับ ผมไม่รู้ความเคลื่อนไหวอะไรเลยในด้านนี้ครับ'

In [209]:
get("ตอนนี้กำลังคุณทดสอบระบบตรวจสอบคำผิกอ่ะนะ")

'ตอนนี้กำลังคุณทดสอบระบบตรวจสอบคำ<คำผิด>ผิก</คำผิด>อ่ะนะ'

In [210]:
get("่าี่าหกาืหวายหผ่ืทก หารหปทากวห")

'<คำผิด>่าี่าหกาืหวายหผ่ืทก</คำผิด> หาร<คำผิด>หปทากวห</คำผิด>'

In [211]:
get("ฉันจะไปแจ้งวัฒนะค่ะ คุนจะไปไหมนะค่ะ")

'ฉันจะไปแจ้งวัฒนะค่ะ <คำผิด>คุน</คำผิด>จะไปไหม<คำผิด>นะค่ะ</คำผิด>'

In [212]:
get("ฉันจะไปแจ้งวัฒนะค่ะ คุนจะไปไหม ถ้าไม่ไปคุณช่วยส่งอีเมลมาให้ฉันที่อีเมล์นี้นะค่ะ")

'ฉันจะไปแจ้งวัฒนะค่ะ <คำผิด>คุน</คำผิด>จะไปไหม ถ้าไม่ไปคุณช่วยส่งอีเมลมาให้ฉันที่<คำผิด>อีเมล์</คำผิด>นี้<คำผิด>นะค่ะ</คำผิด>'

In [213]:
get("อารลมณของคุณเมื่อพักร้อนแล้วกลับมาเช็คอีเมล์ที่ทำงานเป็นครั้งแรก.")

'<คำผิด>อารลมณ</คำผิด>ของคุณเมื่อพักร้อนแล้วกลับมาเช็ค<คำผิด>อีเมล์</คำผิด>ที่ทำงานเป็นครั้งแรก.'

In [214]:
get("ทัมรัยยุค่ะ")

'<คำผิด>ทัมรัย</คำผิด><คำผิด>ยุ</คำผิด>ค่ะ'

In [215]:
get("ทักนะค้ะ")

'ทัก<คำผิด>นะค้ะ</คำผิด>'

In [216]:
get("ชื่ออัลไลค่า")

'ชื่อ<คำผิด>อัลไล</คำผิด>ค่า'

In [217]:
get('เด๋วมานะค้ะ')

'<คำผิด>เด๋ว</คำผิด>มา<คำผิด>นะค้ะ</คำผิด>'

In [218]:
get("มินิ่งคราบ")

'<คำผิด>มินิ่ง</คำผิด><คำผิด>คราบ</คำผิด>'

In [219]:
get('"จำไว้ๆ เกิดมาเพื่อทำให้โลกใบนี้งดงาม” - พุทธทาสภิกขุ')

'"จำไว้ๆ เกิดมาเพื่อทำให้โลกใบนี้งดงาม” - พุทธทาสภิกขุ'

In [220]:
get("อนาคต ... ถ้าคนตกงานเพราะระบบอัตโนมัติหมด จำนวนงานใหม่ที่เกิดจากอินเทอร์เน็ตโตไม่ทัน แล้วใครจะมีเงินไปซื้อสินค้าหรือบริการจากธุรกิจที่นำระบบอัตโนมัติมาใช้งานแทนคน ในเมื่อคนก็ตกงานกันเยอะ")

'อนาคต ... ถ้าคนตกงานเพราะระบบอัตโนมัติหมด จำนวนงานใหม่ที่เกิดจากอินเทอร์เน็ตโตไม่ทัน แล้วใครจะมีเงินไปซื้อสินค้าหรือบริการจากธุรกิจที่นำระบบอัตโนมัติมาใช้งานแทนคน ในเมื่อคนก็ตกงานกันเยอะ'

In [221]:
get("นายกูเตอร์เรสกล่าวในรายงานระบุว่า รัฐบาลในประเทศเหล่านี้มักจะตั้งข้อหาที่เกี่ยวข้องกับ ก่อการร้าย หรือไม่ก็เกี่ยวกับ ความมั่นคง เพื่อเป็นเหตุผลในการอ้างความผิดให้กับกลุ่มคนที่ให้ความร่วมมือกับองค์กรต่างชาติในการตรวจสอบการละเมิดสิทธิมนุษยชน....... อ่านต่อได้ที่ : https://www.posttoday.com/world/564115")

'นายกูเตอร์เรสกล่าวในรายงานระบุว่า รัฐบาลในประเทศเหล่านี้มักจะตั้งข้อหาที่เกี่ยวข้องกับ ก่อการร้าย หรือไม่ก็เกี่ยวกับ ความมั่นคง เพื่อเป็นเหตุผลในการอ้างความผิดให้กับกลุ่มคนที่ให้ความร่วมมือกับองค์กรต่างชาติในการตรวจสอบการละเมิดสิทธิมนุษยชน....... อ่านต่อได้ที่ : https://www.posttoday.com/world/564115'

In [222]:
get('หงุดหงิดโว้ยยยย')

'หงุดหงิดโว้ย<คำผิด>ยยย</คำผิด>'

In [223]:
get('ลนนะลง หั้ยชัยพาสาทัย หัยถูกต้องนะค่ะ 🌝')

'ลนนะลง <คำผิด>หั้ย</คำผิด><คำผิด>ชัย</คำผิด><คำผิด>พาสา</คำผิด><คำผิด>ทัย</คำผิด> <คำผิด>หัย</คำผิด>ถูกต้อง<คำผิด>นะค่ะ</คำผิด> 🌝'

In [224]:
get("เคี้ยวกุดปะ")

'เคี้ยวกุดปะ'

In [225]:
get("ทามรัยข๊' ทีลั๊'คค บอกเล'าา โน่ยเส๊'")

"<คำผิด>ทาม</คำผิด><คำผิด>รัย</คำผิด><คำผิด>ข๊'</คำผิด> <คำผิด>ทีลั๊'คค</คำผิด> บอก<คำผิด>เล'าา</คำผิด> โน่ยเส๊'"

In [226]:
get("กัวแร้วจร้าาาาาา")

'<คำผิด>กัว</คำผิด><คำผิด>แร้ว</คำผิด><คำผิด>จร้าาาาาา</คำผิด>'

In [227]:
get("รั๊คสุ้ดๆอร้ะะะะ")

'<คำผิด>รั๊คสุ้ด</คำผิด>ๆ<คำผิด>อร้ะะะะ</คำผิด>'

In [228]:
get("ทำรัยยุค่ะ")

'<คำผิด>ทำรัย</คำผิด><คำผิด>ยุ</คำผิด>ค่ะ'

In [229]:
get("อัลไลหร๊")

'<คำผิด>อัลไล</คำผิด><คำผิด>หร๊</คำผิด>'

In [230]:
get("ปัยดีณะคะ")

'<คำผิด>ปัย</คำผิด>ดี<คำผิด>ณะคะ</คำผิด>'

In [231]:
get("รับได้บางคำแต่ไม่ทั้งหมด​ ไม่ได้ดูถูกศัพท์วัยรุ่น​ ไม่ได้เป็นคนหัวโบราณ​ แต่ไม่อยากให้ภาษาบ้านตัวเองวิบัตินะคับ")

'รับได้บางคำแต่ไม่ทั้งหมด\u200b ไม่ได้ดูถูกศัพท์วัยรุ่น\u200b ไม่ได้เป็นคนหัวโบราณ\u200b แต่ไม่อยากให้ภาษาบ้านตัวเองวิบัตินะคับ'

In [232]:
get("แต่ของพี่ เอาเข้า dict อยู่นะ ละก็แก้คำผิด > <")

'แต่ของพี่ เอาเข้า dict อยู่นะ ละก็แก้คำผิด > <'

In [233]:
get("ประยุทธ์ ลั่นปม “นาฬิกาเพื่อน” ของ ประวิตร เป็น “เรื่องเล็ก ๆ น้อย” ขอให้จบ ๆ ไป ย้อนถามทีคนหนีคดีทำไมไม่ไปดู หลังไทยถูกปรับลดอันดับคอร์รัปชั่น")

'ประยุทธ์ ลั่นปม “นาฬิกาเพื่อน” ของ ประวิตร เป็น “เรื่องเล็ก ๆ น้อย” ขอให้จบ ๆ ไป ย้อนถามทีคนหนีคดีทำไมไม่ไปดู หลังไทยถูกปรับลดอันดับคอร์รัปชั่น'

In [234]:
get("หยุดใช้ QR code แล้วใช้ short url แทน อะไรๆก็ใช้แต่ QR code แม้แต่กิจกรรมของมหาลัย โพสต์เชิญชวนเข้ากิจกรรมหลายอันแต่เว็บสมัครเป็น QR code อยู่ในรูป จะให้ใช้แต่มือถือ ไม่เอาคอม แถมถ้าอยู่ในมือถือต้องเอารูปนั้นไปผ่านแอพสแกนถอดลิงค์จากรูปอีก เจอหลายงานหงุดหงิด เลยไม่สมัครไปหลายงาน")

'หยุดใช้ QR code แล้วใช้ short url แทน อะไรๆก็ใช้แต่ QR code แม้แต่กิจกรรมของมหาลัย โพสต์เชิญชวนเข้ากิจกรรมหลายอันแต่เว็บสมัครเป็น QR code อยู่ในรูป จะให้ใช้แต่มือถือ ไม่เอาคอม แถมถ้าอยู่ในมือถือต้องเอารูปนั้นไปผ่านแอพสแกนถอดลิงค์จากรูปอีก เจอหลายงานหงุดหงิด เลยไม่สมัครไปหลายงาน'

In [235]:
get("มทส. ไม่มีใครทำแผนที่ AGPS ให้ Mozilla เลยหรอ ?")

'มทส. ไม่มีใครทำแผนที่ AGPS ให้ Mozilla เลยหรอ ?'

In [236]:
get("ตามรอยกรุงเทพแล้วหนองคาย ... ข้อมูลจากดาวเทียมเชื่อถือได้ไหม ?")

'ตามรอยกรุงเทพแล้วหนองคาย ... ข้อมูลจากดาวเทียมเชื่อถือได้ไหม ?'

In [237]:
get("จะเลือกตั้งแล้ว ไปดูข้อมูลเปิดด้านการเลือกตั้ง ที่ กกต. ร่วมกับ สพร. ทำระบบ ตามลิ้งค์ข้างล่างนี้เลย")

'จะเลือกตั้งแล้ว ไปดูข้อมูลเปิดด้านการเลือกตั้ง ที่ กกต. ร่วมกับ สพร. ทำระบบ ตามลิ้งค์ข้างล่างนี้เลย'

In [238]:
get("โซฟาห้องโปรเจคสาขา CIS ช่างนอนหลับสบาย")

'โซฟาห้องโปรเจคสาขา CIS ช่างนอนหลับสบาย'

In [239]:
get("แถลกานล์สภานักศึกษามหาวิทยาลัยเชียงใหม่")

'<คำผิด>แถลกานล์</คำผิด>สภานักศึกษามหาวิทยาลัยเชียงใหม่'

In [240]:
get("หนึ่งวันในดาวเสาร์คือ 10 ชั่วโมง 33 นาที 38 วินาที")

'หนึ่งวันในดาวเสาร์คือ 10 ชั่วโมง 33 นาที 38 วินาที'

In [241]:
get("รับงานพิสูจน์เอกสารนะค่ะ")

'รับงานพิสูจน์เอกสาร<คำผิด>นะค่ะ</คำผิด>'

In [242]:
get("เพราะฉันลืมชื่อเขานะค่ะ")

'เพราะฉันลืมชื่อเขา<คำผิด>นะค่ะ</คำผิด>'

In [243]:
get("เขาปลูกกาแฟบนภูเขสูงส่นวใหญ่เป็นกาแฟดิบ")

'เขาปลูกกาแฟบน<คำผิด>ภูเข</คำผิด>สูง<คำผิด>ส่นว</คำผิด>ใหญ่เป็นกาแฟดิบ'

In [244]:
get("ภูเขบไฟระเบิดที่คาโงชิมะ")

'<คำผิด>ภูเขบ</คำผิด>ไฟระเบิดที่คาโงชิมะ'

In [245]:
get("ไม่ว่าจะฝนตก น้ำท่วม แผ่นดินไหว ภูเขไฟระเบิด มหิดลก้อยังสอบเหมือนเดิม!!")

'ไม่ว่าจะฝนตก น้ำท่วม แผ่นดินไหว <คำผิด>ภูเข</คำผิด>ไฟระเบิด มหิดล<คำผิด>ก้อ</คำผิด>ยังสอบเหมือนเดิม!!'

In [246]:
get("โอวันตินภูเขสไฟ")

'โอวันติน<คำผิด>ภูเขส</คำผิด>ไฟ'

In [247]:
get("มึงจะทำไมกุล่ะ")

'มึงจะทำไม<คำผิด>กุ</คำผิด>ล่ะ'

In [248]:
get("สำหรับอาจารย์ปละพี่ๆศิษย์นะครับ")

'สำหรับอาจารย์<คำผิด>ปละ</คำผิด>พี่ๆศิษย์นะครับ'

In [249]:
get("ช่าย...ตรูไม่โอเคแร้ว")

'<คำผิด>ช่าย</คำผิด>...<คำผิด>ตรู</คำผิด>ไม่โอเค<คำผิด>แร้ว</คำผิด>'

In [250]:
get("ช่าย เสียวมากอะ 2 ตอบกลับ 0 รีทวีต 1 ชื่นชอบ")

'<คำผิด>ช่าย</คำผิด> เสียวมากอะ 2 ตอบกลับ 0 รีทวีต 1 ชื่นชอบ'

In [251]:
get("จีบเธอได้มะ ในฐานะคนที่ชอบ")

'จีบเธอได้<คำผิด>มะ</คำผิด> ในฐานะคนที่ชอบ'

In [252]:
get("ช่าย​ผ่องนั้นแหละน้ำหวาน​ คู่กับน้องอิน​ 😂😂😂")

'<คำผิด>ช่าย</คำผิด>\u200bผ่องนั้นแหละน้ำหวาน\u200b คู่กับน้องอิน\u200b 😂😂😂'

In [253]:
get("ไม่อยากจะเมาท์ เลยจริม จริมม")

'ไม่อยากจะ<คำผิด>เมาท์</คำผิด> เลย<คำผิด>จริม</คำผิด> <คำผิด>จริมม</คำผิด>'

In [254]:
get("เกี่ยวววววววว")

'<คำผิด>เกี่ยวววววววว</คำผิด>'

In [255]:
get("เหมือนมึงกับกุอะ แย่งห้องน้ำกันเก่ง")

'เหมือนมึงกับ<คำผิด>กุ</คำผิด>อะ แย่งห้องน้ำกันเก่ง'

In [256]:
get("ช่วงนี้แกหิวบ่อยช่ะป่ะ")

'ช่วงนี้แกหิวบ่อย<คำผิด>ช่ะ</คำผิด><คำผิด>ป่ะ</คำผิด>'

In [257]:
get("คนน่าร้ากกก ทำอะไรนะ")

'คนน่า<คำผิด>ร้ากกก</คำผิด> ทำอะไรนะ'

In [258]:
get("ไม่ไหวแล้วววววน่ารักเกินไปอร้าก")

'ไม่ไหวแล้ว<คำผิด>วววว</คำผิด>น่ารักเกินไป<คำผิด>อ</คำผิด><คำผิด>ร้าก</คำผิด>'

In [259]:
get("อันนี้น่าร้ากพี่อี้แกล้งจะเปิดเสื้อคยองแล้วน้องร้องเสียงหลงเลยเอ็นดูการรวบชายเสื้อมากุมไว้จะได้ไม่มีใครถกขึ้นได้อ่ะน้องงงง ")

'อันนี้น่า<คำผิด>ร้าก</คำผิด>พี่อี้แกล้งจะเปิดเสื้อคยองแล้วน้องร้องเสียงหลงเลยเอ็นดูการรวบชายเสื้อมากุมไว้จะได้ไม่มีใครถกขึ้นได้อ่ะ<คำผิด>น้องงงง</คำผิด> '

In [260]:
get("น่าร้ากชาม้าดดดดดดด เจ้าหัวใจของพรี่")

'น่า<คำผิด>ร้าก</คำผิด><คำผิด>ชา</คำผิด><คำผิด>ม้าดดดดดดด</คำผิด> เจ้าหัวใจของพรี่'

In [261]:
get("ตัวอย่างประโยคที่พิมผิด")

'ตัวอย่างประโยคที่<คำผิด>พิม</คำผิด>ผิด'

In [262]:
get("พี่พิมรีบทำงานสิ")

'พี่พิมรีบทำงานสิ'

In [263]:
get("เขาพิมงานผิดพลาดทำให้เกิดความเสียหาย")

'เขาพิมงานผิดพลาดทำให้เกิดความเสียหาย'

In [264]:
get("สวัสดีตอนเช้า คิดเถิงแต่เช้าเลย")

'สวัสดีตอนเช้า คิด<คำผิด>เถิง</คำผิด>แต่เช้าเลย'

In [265]:
get("คิดเถิ๊ง คิดเถิง คิดเถิ๊ง คิดเถิง 🥰🥰🥰")

'คิด<คำผิด>เถิ๊ง</คำผิด> คิด<คำผิด>เถิง</คำผิด> คิด<คำผิด>เถิ๊ง</คำผิด> คิด<คำผิด>เถิง</คำผิด> 🥰🥰🥰'

In [266]:
get("แจ่มสุดๆ ขอบคุณครับโผม")

'แจ่มสุดๆ ขอบคุณครับ<คำผิด>โผม</คำผิด>'

In [267]:
get("จากโพสท์ล่าสุดที่น้องพูดเรื่องตัดผมคือเริ่มต้นชีวิตใหม่")

'จาก<คำผิด>โพสท์</คำผิด>ล่าสุดที่น้องพูดเรื่องตัดผมคือเริ่มต้นชีวิตใหม่'

In [268]:
get("เกียดท่าโพสท์55555555555555555555555")

'<คำผิด>เกียด</คำผิด>ท่า<คำผิด>โพสท์</คำผิด>55555555555555555555555'

In [269]:
get("จิงใช่ไหม")

'<คำผิด>จิง</คำผิด>ใช่ไหม'

In [270]:
get("อิเหี้ยยย คอนเสิต บรูโน มาร์ เก้าอี้ 4000 ได้นั่งเก้าอี้นี้จริงเหรอวะ เขร้! ")

'อิ<คำผิด>เหี้ยยย</คำผิด> <คำผิด>คอนเสิต</คำผิด> บรูโน มาร์ เก้าอี้ 4000 ได้นั่งเก้าอี้นี้จริงเหรอวะ เขร้! '

In [271]:
get("มีขึ้นสเตจมั้ย แฟนมีต ไฮทัช คอนเสิต ต้องมีแฟนช้านแท่งไฟอะไรมั้ย งงไปหมดแล้วจ่ะพี่จ๋า ")

'มีขึ้น<คำผิด>สเตจ</คำผิด>มั้ย แฟนมีต ไฮทัช <คำผิด>คอนเสิต</คำผิด> ต้องมีแฟนช้านแท่งไฟอะไรมั้ย งงไปหมดแล้ว<คำผิด>จ่ะ</คำผิด>พี่จ๋า '

In [272]:
get("คอนเสิต ดีวีดีคอน จับมือล้านจังหวัดดดดเ")

'<คำผิด>คอนเสิต</คำผิด> ดีวีดีคอน จับมือล้านจังหวัด<คำผิด>ดดดเ</คำผิด>'

In [273]:
get("นั่งรถผ่านบริเวณที่เป็นเนินสูงๆ เห็นบรรยากาศรอบ ๆ เป็นเหมือนหมอก แต่ที่จริงคือหมอกฝุ่น ... ไม่น่าเชื่อว่า ขอ่่ก่นจะมาถึงจุดนี้")

'นั่งรถผ่านบริเวณที่เป็นเนินสูงๆ เห็นบรรยากาศรอบ ๆ เป็นเหมือนหมอก แต่ที่จริงคือหมอกฝุ่น ... ไม่น่าเชื่อว่า <คำผิด>ขอ่่</คำผิด>ก่นจะมาถึงจุดนี้'

In [274]:
get("จะเบอร์เกอร์หรือจะเฟรนช์ฟรายส์ ส่งฟรีทุกอย่างจาก Burger King ของอร่อยรออยู่น้า หิวแล้วสั่งเลย")

'จะเบอร์เกอร์หรือจะเฟรนช์ฟรายส์ ส่งฟรีทุกอย่างจาก Burger King ของอร่อยรออยู่น้า หิวแล้วสั่งเลย'

In [275]:
get("หนังสืออ่านด้ดี")

'หนังสืออ่าน<คำผิด>ด้</คำผิด>ดี'

In [276]:
get("ผมชอบที่เเมวกินปลาทู่")

'ผมชอบที่<คำผิด>เเมว</คำผิด>กิน<คำผิด>ปลาทู่</คำผิด>'

In [277]:
get("น่ารักอีกเเล้วผมเลยกินปลาคับอิอิน่าลักกกกจังขอใหมีความสุก")

'น่ารักอีก<คำผิด>เเล้ว</คำผิด>ผมเลยกินปลาคับอิอิน่า<คำผิด>ลักกกก</คำผิด>จังขอ<คำผิด>ให</คำผิด>มีความสุก'

In [278]:
get("ไม่มีรถยยเดินทางเลย")

'ไม่มี<คำผิด>รถยย</คำผิด>เดินทางเลย'

In [279]:
get("ทำใ้การค้นหามันตรงขึ้นเยอะ")

'<คำผิด>ทำใ้</คำผิด>การค้นหามันตรงขึ้นเยอะ'

In [280]:
get("ตอนทำเหนเดินเชิดคอยาวเปนยีราฟยุ")

'ตอนทำ<คำผิด>เหน</คำผิด>เดินเชิดคอยาว<คำผิด>เปน</คำผิด>ยีราฟ<คำผิด>ยุ</คำผิด>'